# Multistage Stochastic Unit Commitment Problem

In [43]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg

from sddip import config, utils


## Parameters

In [44]:
test_case_raw_dir = "WB3/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")
scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\s+")
branch_df = pd.read_csv(branch_file_raw, delimiter="\s+")
gen_df = pd.read_csv(gen_file_raw, delimiter="\s+")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\s+")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\s+")

In [45]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

zeros_col = np.zeros((1,ptdf.shape[1]))

ptdf = np.insert(ptdf, ref_bus-1, zeros_col, axis=1)

ptdf

array([[ 0.00000000e+00, -1.00000000e+00, -1.00000000e+00],
       [ 0.00000000e+00, -1.86387445e-16, -1.00000000e+00]])

In [46]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)

# TODO Adjust penalty
penalty = 10000
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

prob = []
p_d = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [47]:
ptdf

array([[ 0.00000000e+00, -1.00000000e+00, -1.00000000e+00],
       [ 0.00000000e+00, -1.86387445e-16, -1.00000000e+00]])

In [48]:
gens_at_bus

[[0], [], []]

In [49]:
# prob[t][n]
# Probability of realization n at stage t
prob

[[1.0], [0.5, 0.5], [0.5, 0.5]]

In [50]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
p_d

[[[0.0, 62.32083260934583, 34.305091739820405]],
 [[0.0, 120.02999105714431, 68.8178383561837],
  [0.0, 131.97491816753634, 68.15322777275095]],
 [[0.0, 127.01202751854132, 65.64150717257031],
  [0.0, 130.3279223545401, 69.6940248570091]]]

In [51]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
ex_pd

[array([ 0.        , 62.32083261, 34.30509174]),
 array([  0.        , 126.00245461,  68.48553306]),
 array([  0.        , 128.66997494,  67.66776601])]

## Optimization

### Variables

In [52]:
model = gp.Model("MSUC")

x = {}
y = {}
s_up = {}
s_down = {}
ys_p = {}
ys_n = {}

# TODO Set startup and shutdown costs
# suc = [10]*n_gens
# sdc = [10]*n_gens

for t in range(n_stages):
    for n in range(n_nodes_per_stage[t]):
        for g in range(n_gens):
            x[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"x_{t+1}_{n+1}_{g+1}")
            y[t,n,g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"y_{t+1}_{n+1}_{g+1}")
            s_up[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_up_{t+1}_{n+1}_{g+1}")
            s_down[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_down_{t+1}_{n+1}_{g+1}")
        ys_p[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_p_{t+1}_{n+1}")
        ys_n[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_n_{t+1}_{n+1}")

model.update()

### Constraints

In [53]:
# Objective
obj = gp.quicksum(prob[t][n]*(gc[g]*y[t,n,g] + suc[g]*s_up[t,n,g] + sdc[g]*s_down[t,n,g] + penalty*(ys_p[t,n] + ys_n[t,n])) 
                    for t in range(n_stages)
                    for n in range(n_nodes_per_stage[t])
                    for g in range(n_gens))

model.setObjective(obj)


# Balance constraints
model.addConstrs((gp.quicksum(y[t,n,g] for g in range(n_gens)) + ys_p[t,n] - ys_n[t,n] == gp.quicksum(p_d[t][n])
                    for t in range(n_stages)
                    for n in range(n_nodes_per_stage[t])),                    
                    "balance")


# Generator constraints
model.addConstrs((y[t,n,g] >= pg_min[g]*x[t,n,g] 
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(n_nodes_per_stage[t])),
                    "min-generation")

model.addConstrs((y[t,n,g] <= pg_max[g]*x[t,n,g]  
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(n_nodes_per_stage[t])),
                    "max-generation")


# Power flow constraints
for t in range(n_stages):
    for n in range(n_nodes_per_stage[t]):
        line_flows = [gp.quicksum(ptdf[l,b] * (gp.quicksum(y[t,n,g] for g in gens_at_bus[b]) - p_d[t][n][b])
                        for b in range(n_buses)) for l in range(n_lines)]
        model.addConstrs((line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(1)")
        model.addConstrs((-line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(2)")


# Startup shutdown constraints
# t=0
x_init = [0]*n_gens
model.addConstrs((x[0,0,g] - x_init[g] <= s_up[0,0,g] for g in range(n_gens)), "up-down(1)")
model.addConstrs((x[0,0,g] - x_init[g] == s_up[0,0,g] - s_down[0,0,g] for g in range(n_gens)), "up-down(2)")
# t>0
for t in range(1,n_stages):
    for n in range(n_nodes_per_stage[t]):
        model.addConstrs((x[t,n,g] - x[t-1,n_prev,g] <= s_up[t,n,g] 
                        for g in range(n_gens) 
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "up-down(1)")
        model.addConstrs((x[t,n,g] - x[t-1,n_prev,g] <= s_up[t,n,g] - s_down[t,n,g] 
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "up-down(2)")


# Ramp rate constraints
# t=0
y_init = [0]*n_gens
model.addConstrs((y[0,0,g] - y_init[g] <= rg_up_max[g] for g in range(n_gens)), "rate-up")
model.addConstrs((y_init[g] - y[0,0,g] <= rg_down_max[g] for g in range(n_gens)), "rate-down(2)")
# t>0
for t in range(1,n_stages):
    for n in range(n_nodes_per_stage[t]):
        model.addConstrs((y[t,n,g] - y[t-1,n_prev,g] <= rg_up_max[g]
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "rate-up")
        model.addConstrs((y[t-1,n_prev,g] - y[t,n,g] <= rg_down_max[g]
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "rate-down")

model.update()
model.display()

Minimize
<gurobi.LinExpr: 2.0 y_1_1_1 + 10000.0 ys_p_1_1 + 10000.0 ys_n_1_1 + y_2_1_1
+ 5000.0 ys_p_2_1 + 5000.0 ys_n_2_1 + y_2_2_1 + 5000.0 ys_p_2_2 + 5000.0 ys_n_2_2
+ y_3_1_1 + 5000.0 ys_p_3_1 + 5000.0 ys_n_3_1 + y_3_2_1 + 5000.0 ys_p_3_2
+ 5000.0 ys_n_3_2>
Subject To
  balance[0,0]: <gurobi.LinExpr: y_1_1_1 + ys_p_1_1 + -1.0 ys_n_1_1> = 96.6259
  balance[1,0]: <gurobi.LinExpr: y_2_1_1 + ys_p_2_1 + -1.0 ys_n_2_1> = 188.848
  balance[1,1]: <gurobi.LinExpr: y_2_2_1 + ys_p_2_2 + -1.0 ys_n_2_2> = 200.128
  balance[2,0]: <gurobi.LinExpr: y_3_1_1 + ys_p_3_1 + -1.0 ys_n_3_1> = 192.654
  balance[2,1]: <gurobi.LinExpr: y_3_2_1 + ys_p_3_2 + -1.0 ys_n_3_2> = 200.022
  min-generation[0,0,0]: <gurobi.LinExpr: y_1_1_1> >= 0
  min-generation[0,1,0]: <gurobi.LinExpr: y_2_1_1> >= 0
  min-generation[0,1,1]: <gurobi.LinExpr: y_2_2_1> >= 0
  min-generation[0,2,0]: <gurobi.LinExpr: y_3_1_1> >= 0
  min-generation[0,2,1]: <gurobi.LinExpr: y_3_2_1> >= 0
  max-generation[0,0,0]: <gurobi.LinExpr: -5000.0 x_1

In [54]:
print(line_flows)

[<gurobi.LinExpr: 200.02194721154922 + 0.0 y_3_2_1>, <gurobi.LinExpr: 69.69402485700913 + 0.0 y_3_2_1>]


### Solve

In [55]:
#model.setParam("OutputFlag",0)

model.optimize()

#model.setParam("OutputFlag",1)

model.printAttr("X")
print(f"Optimal value: {obj.getValue()}")

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 63 rows, 30 columns and 103 nonzeros
Model fingerprint: 0xbab5a2f1
Variable types: 15 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+06]
Found heuristic solution: objective 4874516.5298
Presolve removed 48 rows and 20 columns
Presolve time: 0.00s
Presolved: 15 rows, 10 columns, 30 nonzeros
Found heuristic solution: objective 974.9033060
Variable types: 10 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 2: 974.903 4.87452e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.749033059673e

In [56]:
x_out = [f"x[{t+1},{n+1},{g+1}]:  {x[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
y_out = [f"y[{t+1},{n+1},{g+1}]:  {y[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
s_up_out = [f"s_up[{t+1},{n+1},{g+1}]:  {s_up[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
s_down_out = [f"s_down[{t+1},{n+1},{g+1}]:  {s_down[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]

print("Commitment decisions")
for text in x_out:
    print(f"{text}")

print("Dispatch decisions")
for text in y_out:
    print(f"{text}")

print("Startup decisions")
for text in s_up_out:
    print(f"{text}")

print("Shutdown decisions")
for text in s_down_out:
    print(f"{text}")

Commitment decisions
x[1,1,1]:  1.0
x[2,1,1]:  1.0
x[2,2,1]:  1.0
x[3,1,1]:  1.0
x[3,2,1]:  1.0
Dispatch decisions
y[1,1,1]:  96.62592434916598
y[2,1,1]:  188.84782941332764
y[2,2,1]:  200.1281459402869
y[3,1,1]:  192.65353469111096
y[3,2,1]:  200.02194721154865
Startup decisions
s_up[1,1,1]:  1.0
s_up[2,1,1]:  1.0
s_up[2,2,1]:  1.0
s_up[3,1,1]:  1.0
s_up[3,2,1]:  1.0
Shutdown decisions
s_down[1,1,1]:  0.0
s_down[2,1,1]:  0.0
s_down[2,2,1]:  0.0
s_down[3,1,1]:  0.0
s_down[3,2,1]:  0.0
